In [26]:
import requests
from bs4 import BeautifulSoup
import yfinance as yf
from datetime import datetime
from datetime import timedelta
import pandas as pd
import json
import dateutil.parser
import time
from selenium import webdriver
from functions import (get_sector, changes_from_press, get_content_header, 
                       get_df, normalise_to_index, save_data)
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait 
from selenium.webdriver.support import expected_conditions as EC

past_last_date = False

In [ ]:
data = []
page_num = 0

ticker = 'gd'

sector = get_sector(ticker)

index_ticker = '^GSPC'
# select by value 

stock_hist = get_df(ticker)
index_hist = get_df(index_ticker)

# create break


driver = webdriver.Firefox()
driver.get(f'https://www.gd.com/news/news-feed?page=0&types=Press%20Release')
time.sleep(10)

driver.find_element(By.XPATH, """//*[@id="my-page"]/div/div[4]/div/div[2]/button[1]""").click()
driver.refresh()

while True:
    path = '/html/body/div[1]/div/section/div/div/div/div[3]/div/div/nav/ul/li[8]'
    html = driver.page_source
    soup = BeautifulSoup(html)     
    
    
    articles = soup.find_all('div', attrs={'class': 'news-item row'})
        
    if not articles:
        print('sfsf')
        break
    
   
    for article in articles:
        
        date = article.find('div', attrs={'class':'news__publish-date'}).text
        date = date.replace('Press Release', '').replace('Corporate Statement', '')
        date = date.lstrip().rstrip()
        date = dateutil.parser.parse(date).date()
        title_holder = article.find('h3', attrs={'class':'news__header'})
        
        title = title_holder.find('a').text.lstrip().rstrip() 
        url = title_holder.find('a').get('href')
        url = f'https://www.gd.com{url}'
        
        content = article.find('div', attrs={'class':'news__body'}).text
        if date < datetime(2014, 1, 1).date():
            past_last_date = True
            break        
        
        pct_change = changes_from_press(stock_hist, date, 1)
        if pct_change != None:
            index_price, norm_price = normalise_to_index(pct_change, date, index_hist)
        else:
            index_price, norm_price = (None, None)

        data.append([str(date), title, url, content, pct_change, index_price, norm_price])
    
    driver.find_element((By.XPATH, path)).click()
    time.sleep(50)
    if past_last_date == True: break   

save_data(data, ticker, sector)


In [51]:
past_last_date

False